In [1]:
import pandas as pd
import numpy as np

# Preparacion de datos

In [88]:
def get_postulaciones(size=None):
    postulaciones1 = pd.read_csv('data/FiubaHasta15Abril/fiuba_4_postulaciones.csv', nrows =size)
    postulaciones2 = pd.read_csv('data/fiuba_4_postulaciones.csv', nrows =size)
    postulaciones = pd.concat([postulaciones1, postulaciones2]).drop_duplicates(subset=['idpostulante', 'idaviso'], keep='last')
    columns_rename = {'idaviso': 'id_aviso', 'idpostulante': 'id_postulante', 'fechapostulacion': 'fecha_postulacion'}
    postulaciones = postulaciones.rename(columns=columns_rename)
    postulaciones['fecha_postulacion']=pd.to_datetime(postulaciones['fecha_postulacion'])
    return postulaciones

def get_vistas(size=None):
    vistas1 = pd.read_csv('data/FiubaHasta15Abril/fiuba_3_vistas.csv', nrows =size)
    vistas2 = pd.read_csv('data/FiubaDesde15Abril/fiuba_3_vistas.csv', nrows =size)
    vistas3 = pd.read_csv('data/fiuba_3_vistas.csv', nrows =size)
    vistas = pd.concat([vistas1, vistas2, vistas3])
    vistas_sumarizadas = vistas.groupby(['idpostulante', 'idAviso'], as_index=False)['timestamp'].count()
    columns_rename = {'idAviso': 'id_aviso', 'idpostulante': 'id_postulante', 'timestamp': 'visualizaciones'}
    vistas_sumarizadas = vistas_sumarizadas.rename(columns=columns_rename)
    
    return vistas_sumarizadas

def get_avisos_detalle():
    avisos1 = pd.read_csv('data/fiuba_6_avisos_detalle.csv')
    avisos2 = pd.read_csv('data/FiubaDesde15Abril/fiuba_6_avisos_detalle.csv')
    avisos3 = pd.read_csv('data/FiubaHasta15Abril/fiuba_6_avisos_detalle.csv')
    avisos4 = pd.read_csv('data/fiuba_6_avisos_detalle_missing_nivel_laboral.csv')        
    avisos_detalle = pd.concat([avisos1,avisos2,avisos3,avisos4]).drop_duplicates(subset=['idaviso'], keep='last').reset_index(drop=True)
    columns_rename = {'idpostulante': 'id_postulante', 'idaviso': 'id_aviso'}
    avisos_detalle = avisos_detalle.rename(columns=columns_rename)
    to_nivel_laboral_nro = {'Senior / Semi-Senior' : 2, 'Junior':1, 'Otro':0,
       'Jefe / Supervisor / Responsable':3,
       'Gerencia / Alta Gerencia / Dirección':4}
    to_tipo_trabajo_nro={'Full-time':0, 'Part-time':1, 'Teletrabajo':2, 'Por Horas':3, 'Pasantia':4,
       'Temporario':5, 'Por Contrato':6, 'Fines de Semana':7, 'Primer empleo':8,
       'Voluntario':9}
    to_nombre_area_numero = pd.Series(avisos_detalle['nombre_area'].unique()).to_dict()
    to_nombre_area_numero  = {v: k for k, v in to_nombre_area_numero.items()}
    avisos_detalle['nivel_laboral']= avisos_detalle['nivel_laboral'].map(to_nivel_laboral_nro)
    avisos_detalle['tipo_de_trabajo'] = avisos_detalle['tipo_de_trabajo'].map(to_tipo_trabajo_nro)
    avisos_detalle['nombre_area'] = avisos_detalle['nombre_area'].map(to_nombre_area_numero)
    
    return avisos_detalle

def get_year_of_birth(postulantes_genero_edad):
    return (pd.to_datetime
            (postulantes_genero_edad['fechanacimiento'], errors='coerce', format='%Y-%m-%d')
            .dt.year)

def get_age(yearOfBirth):
    return 2018 - yearOfBirth
    
def get_age_range(yearOfBirth):
    age = get_age(yearOfBirth)
    if(age<25): return 'Entre 18 y 24'
    if(age<30): return 'Entre 25 y 30'
    if(age<35): return 'Entre 30 y 35'
    if(age<40): return 'Entre 35 y 40'
    if(age<45): return 'Entre 40 y 45'
    if(age<50): return 'Entre 45 y 50'
    return 'Mayor de 50'

def get_order_for_age_range():
    return ['Entre 18 y 24', 'Entre 25 y 30','Entre 30 y 35','Entre 35 y 40','Entre 40 y 45','Entre 45 y 50', 'Mayor de 50']

def get_postulantes_nivel_educativo_para(path):
    postulantes_nivel_educativo = pd.read_csv(path)
    columns_rename = {'idpostulante': 'id_postulante', 'nombre': 'formacion_postulante', 'estado': 'estado_formacion_postulante'}
    postulantes_nivel_educativo=postulantes_nivel_educativo.rename(columns=columns_rename)
    formacion_to_number={'Secundario' : 10, 'Otro': 20, 'Terciario/Técnico' : 30, 'Universitario' : 40, 'Posgrado' : 50,
    'Master' : 50, 'Doctorado' : 50}
    postulantes_nivel_educativo['formacion_postulante_numero']=postulantes_nivel_educativo['formacion_postulante'].map(formacion_to_number);
    estado_to_number = {'En Curso': 4, 'Abandonado': 0, 'Graduado': 8}
    postulantes_nivel_educativo['estado_formacion_postulante_numero']=postulantes_nivel_educativo['estado_formacion_postulante'].map(estado_to_number)
    postulantes_nivel_educativo['nivel_educativo_postulante_numero'] = postulantes_nivel_educativo['formacion_postulante_numero'] + postulantes_nivel_educativo['estado_formacion_postulante_numero']
    postulantes_nivel_educativo['nivel_educativo_postulante_texto'] = postulantes_nivel_educativo['formacion_postulante'] + ' - ' + postulantes_nivel_educativo['estado_formacion_postulante']
    relevant_columns = ['id_postulante','nivel_educativo_postulante_texto', 'nivel_educativo_postulante_numero']
    postulantes_nivel_educativo = postulantes_nivel_educativo[relevant_columns]
    grouped=postulantes_nivel_educativo.groupby(['id_postulante']).agg({'nivel_educativo_postulante_numero':['max']}) 
    df=grouped.reset_index()
    df.columns = ['id_postulante', 'maximo_nivel_educativo_postulante']
    return df

def get_postulantes_nivel_educativo():
    postulantes1 = get_postulantes_nivel_educativo_para('data/fiuba_1_postulantes_educacion.csv')
    postulantes2 = get_postulantes_nivel_educativo_para('data/FiubaDesde15Abril/fiuba_1_postulantes_educacion.csv')
    postulantes3 = get_postulantes_nivel_educativo_para('data/FiubaHasta15Abril/fiuba_1_postulantes_educacion.csv')
    return pd.concat([postulantes1,postulantes2,postulantes3]).drop_duplicates(subset=['id_postulante'], keep='last').reset_index(drop=True)

def get_postulantes_genero_edad():
    postulantes1 = pd.read_csv('data/fiuba_2_postulantes_genero_y_edad.csv')
    postulantes2 = pd.read_csv('data/FiubaDesde15Abril/fiuba_2_postulantes_genero_y_edad.csv')
    postulantes3 = pd.read_csv('data/FiubaHasta15Abril/fiuba_2_postulantes_genero_y_edad.csv')
    postulantes_genero_edad = pd.concat([postulantes1,postulantes2,postulantes3]).drop_duplicates(subset=['idpostulante'], keep='last').reset_index(drop=True)
    postulantes_genero_edad['año_nacimiento_postulante']=get_year_of_birth(postulantes_genero_edad)
    postulantes_genero_edad['edad_postulante']=postulantes_genero_edad['año_nacimiento_postulante'].map(get_age, na_action=None)
    postulantes_genero_edad['rango_edad_postulante']=postulantes_genero_edad['año_nacimiento_postulante'].map(get_age_range, na_action=None)
    columns_rename = {'idpostulante': 'id_postulante', 'fechanacimiento': 'fecha_nacimiento_postulante', 'sexo': 'genero_postulante'}
    postulantes_genero_edad = postulantes_genero_edad.rename(columns=columns_rename)
    postulantes_genero_edad = postulantes_genero_edad[['id_postulante', 'genero_postulante', 'fecha_nacimiento_postulante', 'edad_postulante', 'rango_edad_postulante']]
    postulantes_genero_edad['genero_postulante'] = postulantes_genero_edad['genero_postulante'].map({'FEM': 0, 'MASC': 1, 'NO_DECLARA': 2})
    return postulantes_genero_edad

def get_postulantes_limpios():
    postulantes = pd.merge(get_postulantes_genero_edad(), get_postulantes_nivel_educativo(), on='id_postulante', how='outer')
    order_for_columns = ['id_postulante','edad_postulante', 'genero_postulante', 'maximo_nivel_educativo_postulante']
    return postulantes[order_for_columns]

def get_detalle_postulaciones(size=None):
    postulaciones = get_postulaciones(size)
    avisos = get_avisos_detalle()
    postulantes = get_postulantes_limpios()
    detalle_postulaciones = pd.merge(postulantes, postulaciones, on='id_postulante', how='inner') 
    detalle_postulaciones = pd.merge(detalle_postulaciones, avisos, on='id_aviso', how='inner')
    
    return detalle_postulaciones

def get_detalle_vistas(size=None):
    vistas = get_vistas(size)
    avisos = get_avisos_detalle()
    postulantes = get_postulantes_limpios()
    detalle_vistas = pd.merge(postulantes, vistas, on='id_postulante', how='inner') 
    detalle_vistas = pd.merge(detalle_vistas, avisos, on='id_aviso', how='inner')
    return detalle_vistas

def x_entrenamiento():
    return ['edad_postulante', 'genero_postulante', 'maximo_nivel_educativo_postulante', 'nivel_laboral', 'visualizaciones', 'nombre_area', 'titulo']

def y_entrenamiento():
    return ['sepostulo']

def columnas_relevantes_test_data():
    return ['id','id_aviso','id_postulante'] + x_entrenamiento()

def get_test_data():
    tests = pd.read_csv('data/Test/test_final_100k.csv')
    columns_rename = {'idpostulante': 'id_postulante', 'idaviso': 'id_aviso'}
    tests = tests.rename(columns=columns_rename)
    tests = pd.merge(tests, get_avisos_detalle(), on='id_aviso', how='left')
    tests = pd.merge(tests, get_postulantes_limpios(), on='id_postulante', how='left')
    return tests[columnas_relevantes_test_data()]

def get_default_null_values(df):
    return {'edad_postulante':int(df['edad_postulante'].mean()), 'genero_postulante':int(0), 'maximo_nivel_educativo_postulante':int(df['maximo_nivel_educativo_postulante'].mean()), 'nivel_laboral':int(0), 'visualizaciones':int(0), 'nombre_area':int(0), 'titulo':string(0)}


def get_test_data_clean():
    tests = get_test_data()
    tests = tests.fillna(value=get_default_null_values(tests))
    return tests

def get_datos_entrenamiento(size=None):
    postulaciones_aplicadas = get_detalle_postulaciones(size)
    columnas_relevantes = x_entrenamiento() + y_entrenamiento()
    postulaciones_aplicadas['sepostulo'] = True
    postulaciones_no_aplicadas = get_detalle_vistas(size)
    postulaciones_no_aplicadas['sepostulo'] = False

    return postulaciones_aplicadas.append(postulaciones_no_aplicadas).drop_duplicates(subset=['id_aviso', 'id_postulante'], keep='first')[columnas_relevantes].dropna()


In [89]:
set_entrenamiento = get_datos_entrenamiento(10000)

In [90]:
set_entrenamiento.head()

,edad_postulante,genero_postulante,maximo_nivel_educativo_postulante,nivel_laboral,visualizaciones,nombre_area,titulo,sepostulo
0,44.0,1.0,58.0,3.0,1.0,78,Jefe de Gestión de la Demanda,False
1,44.0,1.0,58.0,2.0,1.0,2,Office Manager,False
2,33.0,1.0,58.0,2.0,1.0,2,Office Manager,False
3,38.0,1.0,38.0,2.0,1.0,2,Office Manager,False
4,28.0,1.0,48.0,2.0,1.0,2,Office Manager,False


In [91]:
from sklearn.feature_extraction import FeatureHasher
h = FeatureHasher(n_features=20, input_type='string')
x = h.transform(set_entrenamiento['titulo'])
set_entrenamiento['titulo'] = list(x.toarray().tolist())

In [92]:
set_entrenamiento.head()

,edad_postulante,genero_postulante,maximo_nivel_educativo_postulante,nivel_laboral,visualizaciones,nombre_area,titulo,sepostulo
0,44.0,1.0,58.0,3.0,1.0,78,"[0.0, -1.0, 0.0, 1.0, -2.0, 0.0, 1.0, 0.0, 0.0...",False
1,44.0,1.0,58.0,2.0,1.0,2,"[0.0, -1.0, 0.0, 2.0, -1.0, 1.0, 0.0, 0.0, 0.0...",False
2,33.0,1.0,58.0,2.0,1.0,2,"[0.0, -1.0, 0.0, 2.0, -1.0, 1.0, 0.0, 0.0, 0.0...",False
3,38.0,1.0,38.0,2.0,1.0,2,"[0.0, -1.0, 0.0, 2.0, -1.0, 1.0, 0.0, 0.0, 0.0...",False
4,28.0,1.0,48.0,2.0,1.0,2,"[0.0, -1.0, 0.0, 2.0, -1.0, 1.0, 0.0, 0.0, 0.0...",False


# Armado de los sets de test y de entrenamiento

In [93]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(set_entrenamiento, test_size=0.2)


In [69]:
train['sepostulo'].value_counts()

False    11180
Name: sepostulo, dtype: int64

In [54]:
test['sepostulo'].value_counts()

False    2795
Name: sepostulo, dtype: int64

In [64]:
test.head()

,edad_postulante,genero_postulante,maximo_nivel_educativo_postulante,nivel_laboral,visualizaciones,nombre_area,titulo,sepostulo
3865,21.0,1.0,28.0,2.0,1.0,12,"[-1.0, -4.0, 2.0, 2.0, -1.0, 5.0, -3.0, 1.0, 0...",False
4353,24.0,1.0,18.0,2.0,1.0,56,"[0.0, -6.0, 5.0, 0.0, 0.0, 0.0, 1.0, 0.0, -2.0...",False
2594,26.0,1.0,48.0,0.0,1.0,2,"[0.0, -6.0, 2.0, -1.0, -3.0, 2.0, 1.0, 0.0, 0....",False
12309,33.0,0.0,44.0,2.0,2.0,9,"[0.0, -7.0, 13.0, 0.0, 0.0, 0.0, 5.0, 2.0, -3....",False
14333,26.0,1.0,48.0,2.0,1.0,6,"[0.0, -13.0, 12.0, 0.0, 0.0, 0.0, 3.0, 2.0, -2...",False


# Entrenamiento

In [62]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [94]:
def get_random_forest_predictor(set_entrenamiento):
    X = set_entrenamiento.loc[:, x_entrenamiento()]
    Y = set_entrenamiento.loc[:, 'sepostulo']
    clf = RandomForestClassifier(n_estimators=15)
    return clf.fit(X['titulo'].tolist(), Y)

predictor = get_random_forest_predictor(train)

# Prueba y resultados 

In [103]:
X = test[x_entrenamiento()]['titulo'].tolist()
result = predictor.predict(X)
np.mean(result == test['sepostulo'])

1.0

# Prueba y resultados para la competencia

In [20]:
def obtener_predicciones(predictor, set_test):
    return predictor.predict(set_test[x_entrenamiento()]).astype(int)

def guardar_resultados(fileName, predictor, set_test):
    result = obtener_predicciones(predictor, set_test)
    set_test['sepostulo'] = result
    set_test[['id','sepostulo']].set_index('id').to_csv(fileName)
    return

set_test =get_test_data_clean()
guardar_resultados('100K ultimo.csv', predictor, set_test)
set_test['sepostulo'].value_counts()

1    64627
0    35373
Name: sepostulo, dtype: int64